In [20]:
import base64
import io
import requests
from requests import Response
from PIL import Image, ImageDraw, ImageFont, ImageFilter
from openai import OpenAI
from pprint import pprint
import google.generativeai as genai
import google.ai.generativelanguage as glm
from dotenv import load_dotenv
import os
import pathlib
import rasterio
from rasterio.windows import Window
from shapely.geometry import box

import geopandas as gpd
import pandas as pd

In [21]:
tamaño_trozo = 100
geo_dict = { 'id' : [], 'geometry' : []}



In [22]:

with rasterio.open("./jilin/imagen_cortada.tif") as src:
    pos_y = 0
    pos_x = 0
    width, height = src.shape
    for y in range(0, height, tamaño_trozo):
        for x in range(0, width, tamaño_trozo):

            window = Window(col_off=x, row_off=y, width=x+tamaño_trozo, height=y+tamaño_trozo)
            bbox = rasterio.windows.bounds(window, src.transform)
            bbox = box(*bbox)
            geo_dict['id'].append(f"{x // 100}-{y // 100}")
            geo_dict['geometry'].append(bbox)

            pos_x = pos_x + 1
        pos_y = pos_y + 1
        pos_x = 0



results = gpd.GeoDataFrame(pd.DataFrame(geo_dict))
results.crs = {'init' :'epsg:32719'}
results.to_file("salida-final", driver="GeoJSON")

print(results)

       id                                           geometry
0     0-0  POLYGON ((316607.253 6431214.091, 316607.253 6...
1     1-0  POLYGON ((317207.253 6431214.091, 317207.253 6...
2     2-0  POLYGON ((317807.253 6431214.091, 317807.253 6...
3     3-0  POLYGON ((318407.253 6431214.091, 318407.253 6...
4     4-0  POLYGON ((319007.253 6431214.091, 319007.253 6...
..    ...                                                ...
195  3-24  POLYGON ((318407.253 6416814.091, 318407.253 6...
196  4-24  POLYGON ((319007.253 6416814.091, 319007.253 6...
197  5-24  POLYGON ((319607.253 6416814.091, 319607.253 6...
198  6-24  POLYGON ((320207.253 6416814.091, 320207.253 6...
199  7-24  POLYGON ((320807.253 6416814.091, 320807.253 6...

[200 rows x 2 columns]


/home/palios/programing/fach-dataton/.venv/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [23]:
results.to_file("shape/results_2.shp")
